In [1]:
from bs4 import BeautifulSoup as bs
#from sklearn.cluster import KMeans
import requests
from pymongo import MongoClient
import pandas as pd
import time
import re
import math
pd.set_option('display.max_colwidth',-1)

In [2]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("italian"))
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('italian')


In [3]:
def cleanData(rawData, lang='italian'):
        tokenizer = RegexpTokenizer(r'\w+')
        stop_words = set(stopwords.words(lang))

        # get words lowercased
        t0 = rawData.lower()
        # remove puctuations
        t1 = tokenizer.tokenize(t0)

        # remove stop words
        t2 =[]
        t2 = [t1[i] for i in range(0,len(t1)) if t1[i] not in stop_words]

        # stemm words
        t3 = [stemmer.stem(t2[i]) for i in range(0, len(t2))]

        # remove nummbers and strings starting with numbers
        t4 = [t3[i] for i in range(0, len(t3)) if t3[i][0].isdigit()==False]

        return(t4)

In [4]:
data=pd.read_csv('df.csv', sep='\t')
url=[]
for i in data.iloc[:,1]:
    url.append(i)

In [5]:
data

,Unnamed: 0,0,1,2,3,4,5
0,0,https://www.immobiliare.it/53131931-Vendita-Bilocale-viale-Italo-Calvino-Roma.html,225.000,2,50.0,1,1
1,1,https://www.immobiliare.it/70147844-Vendita-Trilocale-via-Mortara-5-Roma.html,349.000,3,100.0,1,2
2,2,https://www.immobiliare.it/68902925-Vendita-Attico-Mansarda-via-Taranto-100-Roma.html,459.000,2,85.0,1,A
3,3,https://www.immobiliare.it/70114842-Vendita-Attico-Mansarda-via-Degli-Orti-Di-Galba-Roma.html,850.000,5,175.0,2,A
4,4,https://www.immobiliare.it/69862702-Vendita-Appartamento-via-San-Sotero-Roma.html,1.100.000,5+,225.0,3+,3
5,5,https://www.immobiliare.it/69936424-Vendita-Attico-Mansarda-piazzale-Clodio-12-Roma.html,1.300.000,4,115.0,2,9
6,6,https://www.immobiliare.it/67622989-Vendita-Appartamento-via-San-Marcello-Pistoiese-73-Roma.html,410.000,5,141.0,2,1
7,7,https://www.immobiliare.it/68926129-Vendita-Appartamento-via-Edoardo-Jenner-163-Roma.html,695.000,5+,250.0,3,3
8,8,https://www.immobiliare.it/67706335-immobili_commerciali-ufficio-in-vendita-Roma.html,1.220.000,5+,245.0,2,T
9,9,https://www.immobiliare.it/66064221-Vendita-Appartamento-viale-delle-Mura-Gianicolensi-Roma.html,1.380.000,5+,140.0,2,2


In [8]:
def DescriptionDf(listOfUrl):
    df=pd.DataFrame()
    i=0
    for web in listOfUrl:
        content = requests.get(web)
        soup = bs(content.text, 'html.parser')
        desc = soup.find('div',attrs={'class':'col-xs-12 description-text text-compressed','role':'contentinfo','aria-expanded':'false'}).get_text()               
        desc=re.sub('\n',' ',desc)
        desc=re.sub('_',' ',desc)
        desc=re.sub('\t','',desc)
        x=["announcement "+str(i),web,desc]
        interest=pd.DataFrame([x],columns=["Title","Url","Description"])
        df=pd.concat([df,interest])
        # print(i) for checking
        i+=1
    display(df)
    return(df)

In [9]:
#tr=url[:374]
#Data2=DescriptionDf(tr)

Data2=DescriptionDf(url)    #n 375, n 524, n 634, n 751, n 797, n 809, n 859, n 861, n 862, n 1018,
                            #n 1109, n 1112, n 1217, n 1269, n 1273, n 1298, n 1305.. can't find the web sites

,Title,Url,Description
0,announcement 0,https://www.immobiliare.it/53131931-Vendita-Bilocale-viale-Italo-Calvino-Roma.html,"PAPILLO EUR in elegante complesso residenziale rifinitissimo bilocale composto da soggiorno con angolo cottura, stanza da letto bagno e ampio balcone . con Rifiniture di pregio, pavimenti in parquet / grees, infissi in legno con vetro camera e porte in noce, grate nel salone, riscaldamento termoautonomo con caldaia centralizzata, aria condizionata, videocitofono, porta blindata, serramenti elettrici con chiusura centralizzata, antenna satellitare, isolamento termo acustico, pannelli solari e fotovoltaici , rilevatori elettronici di gas. Tutte le camere sono fornite di impianto antifurto, presa antenna satellitare e presa telefonica. ORARI lunedi chiusi martedi 10:00-17:00 mercoledi 10:00-17:00 giovedi 10:00-17:00 venrdi 10:00-17:00 sabato 10:00-17:00 domenica 10:00-13:00"
0,announcement 1,https://www.immobiliare.it/70147844-Vendita-Trilocale-via-Mortara-5-Roma.html,"Rif: 18102 - RE DI ROMA - Via Mortara. A pochi passi da Piazza Re di Roma. In zona ben servita da mezzi pubblici e attività commerciali di ogni genere, proponiamo all'interno di una strada privata, la vendita di un appartamento in buono stato di 100mq, posto al piano secondo e composto da: ampio salone, cucina abitabile, due camere, servizio e ripostiglio. I dati si intendono corretti ma non garantiti. La presente non costituisce, in nessun caso, documento contrattuale. Inserisci la “via” e il “cap” per ricevere la valutazione immediata della Tua casa! Entra nel nostro sito web: http://www.angeliniproperty.it/valuta-la-tua-casa/ Tutte le novità le trovi sui nostri canali Social: Seguici su INSTAGRAM: https://www.instagram.com/la tua family agency/ Like su FACEBOOK: https://www.facebook.com/angeliniproperty/?ref=bookmarks Iscriviti su YOUTUBE: https://www.youtube.com/channel/UCKYLafeitvlEs6EMft9l-rA"
0,announcement 2,https://www.immobiliare.it/68902925-Vendita-Attico-Mansarda-via-Taranto-100-Roma.html,"San Giovanni, Attico in Vendita, via Taranto libero luminoso ingresso salone camera possibilità seconda camera cucina abitabile bagno. Terrazzatissimo. Dilazioni - Permute. Rif. 1008S - Tel. 06.69353612 Penthouse for Sale, free bright entrance hall living room kitchen bathroom large terrace. libre entrée lumineuse hall d'entrée salon cuisine salle de bain grande terrasse. Ático en venta,, hall de entrada luminoso gratuito sala de estar, cocina, baño, amplia terraza."
0,announcement 3,https://www.immobiliare.it/70114842-Vendita-Attico-Mansarda-via-Degli-Orti-Di-Galba-Roma.html,"Nel quartiere Monteverde, a poche decine di metri da Villa Doria Pamphili, e più precisamente nella silenziosa Via degli Orti di Galba, la Coldwell Banker propone in vendita un ampio e luminoso attico. Posto al quarto ed ultimo piano di una palazzina recentemene ripulita, l'attico si compone di un doppio salone con pavimentazione in marmo e con affacci nel verde dell'attigua villa Pamphili. L'accesso allo spazioso terrazzo è garantito sia dal salone che dall'adiacente ed ampia cucina/tinello. La zona notte, ottimamente distribuita, presenta pavimentazione in parquet ed è composta da un grande disimpegno dotato di un comodo soppalco, tre spaziose camere da letto, due bagni ed un ripostiglio. Completano la proprietà un box doppio ed un'ampia cantina. L'attico è, inoltre, dotato di porta blindata, inferriate e sistema d'allarme, e si presenta in ottime condizioni abitative. L'ottima distribuzione degli spazi interni, la luminosità e la vicinanza con il parco più grande della Capitale, nonché con il Gianicolo, rendono questo immobile ideale per famiglie e per gli amanti del quartiere Monteverde."
0,announcement 4,https://www.immobiliare.it/69862702-Vendita-Appartamento-via-San-Sotero-Roma.html,"Gregorio Settimo - San Damaso Nelle immediate vicinanze del Vaticano, in elegante stabile con servizio di portineria, proponiamo la vendita di un prestigioso appartamento sito al terzo p

**Building the dictionaries**

In [10]:
def TermDictionaries(ColumnOfDerscriptions):
    Term_Id={}
    Inv_Dict={}
    num=0
    for i in ColumnOfDerscriptions:
        app=cleanData(i, lang='italian')
        for w in set(app):
            if w not in Term_Id:
                Inv_Dict[len(Term_Id)]=["announcement"+str(num)]
                Term_Id[w]=len(Term_Id)        
            else:
                Inv_Dict[Term_Id.get(w)].append("announcement"+str(num))
        num+=1
    return(Term_Id,Inv_Dict)

In [11]:
Term_Id,Inv_Dict=TermDictionaries(Data2.loc[:,"Description"])

In [12]:
Term_Id

{'preg': 0,
 'rifinitissim': 1,
 'termoautonom': 2,
 'ampi': 3,
 'orar': 4,
 'elettron': 5,
 'acust': 6,
 'isol': 7,
 'soggiorn': 8,
 'residenzial': 9,
 'cottur': 10,
 'videocitof': 11,
 'pav': 12,
 'gas': 13,
 'compless': 14,
 'serr': 15,
 'cam': 16,
 'fotovolt': 17,
 'pannell': 18,
 'mercoled': 19,
 'centralizz': 20,
 'compost': 21,
 'domen': 22,
 'stanz': 23,
 'eleg': 24,
 'papill': 25,
 'vetr': 26,
 'grees': 27,
 'tutt': 28,
 'chius': 29,
 'ari': 30,
 'bilocal': 31,
 'chiusur': 32,
 'forn': 33,
 'marted': 34,
 'legn': 35,
 'telefon': 36,
 'caldai': 37,
 'eur': 38,
 'lett': 39,
 'antifurt': 40,
 'venrd': 41,
 'bagn': 42,
 'rifinitur': 43,
 'riscald': 44,
 'condizion': 45,
 'rilev': 46,
 'noc': 47,
 'luned': 48,
 'antenn': 49,
 'blind': 50,
 'satellit': 51,
 'angol': 52,
 'salon': 53,
 'elettr': 54,
 'term': 55,
 'infiss': 56,
 'solar': 57,
 'impiant': 58,
 'pres': 59,
 'sab': 60,
 'port': 61,
 'parquet': 62,
 'balcon': 63,
 'grat': 64,
 'camer': 65,
 'gioved': 66,
 'http': 67,
 'ven

In [13]:
Inv_Dict

{0: ['announcement0',
  'announcement14',
  'announcement18',
  'announcement32',
  'announcement38',
  'announcement49',
  'announcement50',
  'announcement56',
  'announcement63',
  'announcement65',
  'announcement67',
  'announcement77',
  'announcement78',
  'announcement81',
  'announcement96',
  'announcement116',
  'announcement135',
  'announcement139',
  'announcement150',
  'announcement154',
  'announcement155',
  'announcement156',
  'announcement157',
  'announcement158',
  'announcement159',
  'announcement163',
  'announcement170',
  'announcement210',
  'announcement215',
  'announcement217',
  'announcement226',
  'announcement235',
  'announcement238',
  'announcement247',
  'announcement248',
  'announcement252',
  'announcement263',
  'announcement267',
  'announcement273',
  'announcement275',
  'announcement276',
  'announcement277',
  'announcement278',
  'announcement281',
  'announcement286',
  'announcement288',
  'announcement296',
  'announcement299',
  'an

In [14]:
def Idf(word,Inv_Dict,Term_Id,Data):
    key=Term_Id[word]
    Idf={}
    n=len(Data)
    Idf=math.log10(n/len(Inv_Dict[key]))
    return Idf

def Tf(word,ann):
    Tf=ann.count(word)/len(ann)
    return Tf        
        
        

In [16]:
matrix=pd.DataFrame()
num=0
col=["Announcement",*Term_Id.keys()]
for i in Data2.loc[:,"Description"]:
    app=cleanData(i, lang='italian')
    score=["announcement"+str(num)]
    for word in Term_Id.keys():
        TfIdf=Tf(word,app)*Idf(word,Inv_Dict,Term_Id,Data2)
        score.append(TfIdf)
    interest=pd.DataFrame([score],columns=col)
    matrix=pd.concat([matrix,interest])
    num+=1
matrix.set_index('Announcement') 


,preg,rifinitissim,termoautonom,ampi,orar,elettron,acust,isol,soggiorn,residenzial,...,minimal,massicc,ora,campagn,maranc,mantegn,pigafett,ambent,campion,avicenn
Announcement,,,,,,,,,,,,,,,,,,,,,
announcement0,0.011786,0.026026,0.0194,0.002726,0.031553,0.026026,0.021845,0.015423,0.007273,0.012138,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
announcement1,0.000000,0.000000,0.0000,0.002045,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
announcement2,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
announcement3,0.000000,0.000000,0.0000,0.005662,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
announcement4,0.000000,0.000000,0.0000,0.004267,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
announcement5,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
announcement6,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008323,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
announcement7,0.000000,0.000000,0.0000,0.004907,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
announcement8,0.000000,0.000000,0.0000,0.002689,0.000000,0.000000,0.010773,0.007606,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
